# Lab 3.08

In this lab, you will be using the Sakila database of movie rentals.

In order to optimize our inventory, we would like to know which films will be rented next month and we are asked to create a model to predict it.
Instructions

- Create a query or queries to extract the information you think may be relevant for building the prediction model. It should include some film features and some rental features. Use the data from 2005.
- Create a query to get the list of films and a boolean indicating if it was rented last month (August 2005). This would be our target variable.
- Read the data into a Pandas dataframe.
- Analyze extracted features and transform them. You may need to encode some categorical variables, or scale numerical variables.
- Create a logistic regression model to predict this variable from the cleaned data.
- Evaluate the results.


## 1. Create a query or queries to extract the information you think may be relevant for building the prediction model. It should include some film features and some rental features. Use the data from 2005.

This is the info I think I need: 

#### (Later relization: film_id and rental_id could have been dropped here, as I don't really need them anymore)

## 2. Create a query to get the list of films and a boolean indicating if it was rented last month (August 2005). This would be our target variable.

SELECT f.film_id, 
       f.title, 
       f.rental_rate,
       f.length,
       f.rating,
       c.name AS category,
       r.rental_id, 
       r.rental_date, 
       r.return_date, 
       CASE WHEN MONTH(r.rental_date) = 8 
            THEN "True"
            ELSE "False"
            END AS rented_last_month 
       FROM film f
JOIN inventory USING(film_id)
JOIN rental r USING(inventory_id)
JOIN film_category fc ON f.film_id = fc.film_id
JOIN category c ON fc.category_id = c.category_id;

(Another later realization: what I do here is to check for every *copy* of the movie whether it was rented last month. But it would make more sense to get that info for every *movie*)

#### HOWEVER: exploring this in MySQL Workbench showed me that every movie, at least one copy has been rented last month - this returns one 'True' and one 'False' row for every movie. So this is not very helpful for regression, to say the least.

## 3. Read the data into a Pandas dataframe.

In [ ]:
# THIS WAS THE FIRST PASS, WHERE I DID IT WRONG


import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass
import numpy as np
password = getpass.getpass()

# get the data
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)
query = '''SELECT f.film_id, 
       f.title, 
       f.rental_rate,
       f.length,
       f.rating,
       c.name AS category,
       r.rental_id, 
       r.rental_date, 
       r.return_date, 
       CASE WHEN MONTH(r.rental_date) = 8 
            THEN "True"
            ELSE "False"
            END AS rented_last_month 
       FROM film f
JOIN inventory USING(film_id)
JOIN rental r USING(inventory_id)
JOIN film_category fc ON f.film_id = fc.film_id
JOIN category c ON fc.category_id = c.category_id;'''

data = pd.read_sql_query(query, engine)
data.shape

In [1]:
# THIS IS THE RIGHT ONE: SAME THING, BUT WITHOUT ALL THE RENTAL DATES AND IDs, AND WITH SELECT DISTINCT

import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass
password = getpass.getpass()

# get the data
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)
query = '''SELECT DISTINCT f.film_id, 
       f.title, 
       f.rental_rate,
       f.length,
       f.rating,
       c.name AS category,
       CASE WHEN MONTH(r.rental_date) = 5
            THEN "True"
            ELSE "False"
            END AS rented_last_month 
       FROM film f
JOIN inventory USING(film_id)
JOIN rental r USING(inventory_id)
JOIN film_category fc ON f.film_id = fc.film_id
JOIN category c ON fc.category_id = c.category_id;'''

data = pd.read_sql_query(query, engine)
data.shape

········


(1644, 7)

This gives 958 unique films, although there are 1000 films in the table 'films'. The reason for this is the joins: there are 42 films in the 'films' table which are not in the inventory.

I'm **sure** there's some more elegant way to only get 958 rows - as it is, I get a rough dataset from SQL and clean it up in Pandas. If I have time (which I doubt I will) I might go back to see if I can get the SQL query perfect right away (saving me a few steps in pandas).

## 4. Analyze extracted features and transform them. You may need to encode some categorical variables, or scale numerical variables.

In [ ]:
data.tail(60)

In [2]:
# Now I want to clean this up: for every movie where a 'True' exists I want to drop the 'False' column. 

# The first way that occurs to me is to generate a new list from all of the rows. 

newlist = []
for i in data.index[:-1]: # I need the slicing to also get the last one working (else it'll iterate further than the last index and break)

    # The first one in a set of two gets a 'True' (there are only sets where one is true, because of SELECT DISTINCT in SQL)
    if data['title'][i] == data['title'][i+1]: 
        newlist.append("True")
        
    # The second one in a set gets passed over - nothing gets added
    elif data['title'][i] == data['title'][i-1]:
        pass
    
    # And if something is neither the first nor the second one in a set, it is itself either true or false:
    else: newlist.append(data['rented_last_month'][i])
        


Note: I'm using "True" and "False" strings rather than Booleans, because I do not need python to interact with the data in any way. Since the imported stuff from SQL was structured as strings, I stuck with that. 

In [3]:
# Next, I'll remove all the duplicate titles from my data: 

data = data.drop_duplicates(subset='title')


In [4]:
# And then, set the 'newlist' with Booleans into the dataframe, as 'rented_last_month'

data['rented_last_month'] = newlist

In [6]:
data.tail()

,film_id,title,rental_rate,length,rating,category,rented_last_month
1637,931,VALENTINE VANISHING,0.99,48,PG-13,Travel,True
1639,977,WINDOW SIDE,2.99,85,R,Travel,False
1640,981,WOLVES DESIRE,0.99,55,NC-17,Travel,False
1641,988,WORKER TARZAN,2.99,139,R,Travel,False
1642,989,WORKING MICROCOSMOS,4.99,74,R,Travel,True


In [7]:
data.dtypes

film_id                int64
title                 object
rental_rate          float64
length                 int64
rating                object
category              object
rented_last_month     object
dtype: object

In [8]:
data.isna().sum()

film_id              0
title                0
rental_rate          0
length               0
rating               0
category             0
rented_last_month    0
dtype: int64

I can drop the film_id and title at this point:

In [9]:
data = data.drop(columns= ['film_id', 'title'])

The I have two categorical columns to (potentially) encode: rating and category.

In [10]:
data.rating.value_counts()

PG-13    213
NC-17    202
R        189
PG       183
G        171
Name: rating, dtype: int64

In [11]:
data.category.value_counts()

Sports         73
Family         67
Foreign        67
Animation      64
Documentary    63
Action         61
Drama          61
New            60
Sci-Fi         59
Children       58
Games          58
Comedy         56
Classics       54
Horror         53
Travel         53
Music          51
Name: category, dtype: int64

In [ ]:
# Note to self: I might want to drop the whole 'category' column if the encoded data becomes too unwieldy.

## X-y split

In [12]:
X = data.drop(columns = ['rented_last_month'])
y = data['rented_last_month']

## Train-test split

In [13]:
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=420)

## Numerical - categorical "split"

In [14]:
X_train_num = X_train.select_dtypes(include = np.number)
X_test_num = X_test.select_dtypes(include = np.number)

X_train_cat = X_train.select_dtypes(include = object)
X_test_cat = X_test.select_dtypes(include = object)

## Normalize numerical data

In [15]:
from sklearn.preprocessing import MinMaxScaler

transformer = MinMaxScaler().fit(X_train_num) 

# Training data

X_train_normalized = transformer.transform(X_train_num)
X_train_norm = pd.DataFrame(X_train_normalized, columns = X_train_num.columns)


In [16]:
# test data


X_test_normalized = transformer.transform(X_test_num)
X_test_norm = pd.DataFrame(X_test_normalized, columns = X_test_num.columns)

## Encode categorical data

In [19]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(drop="first").fit(X_train_cat)

# Training data
encoded_train = encoder.transform(X_train_cat).toarray()

names_train = encoder.get_feature_names_out(['rating', 'category'])
encoded_train_df = pd.DataFrame(data=encoded_train, columns = names_train)
encoded_train_df

,rating_NC-17,rating_PG,rating_PG-13,rating_R,category_Animation,category_Children,category_Classics,category_Comedy,category_Documentary,category_Drama,category_Family,category_Foreign,category_Games,category_Horror,category_Music,category_New,category_Sci-Fi,category_Sports,category_Travel
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
761,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
762,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
763,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
764,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
# test data
encoded_test = encoder.transform(X_test_cat).toarray()

names_test = encoder.get_feature_names_out(['rating', 'category'])
encoded_test_df = pd.DataFrame(data=encoded_test, columns = names_test)
encoded_test_df

,rating_NC-17,rating_PG,rating_PG-13,rating_R,category_Animation,category_Children,category_Classics,category_Comedy,category_Documentary,category_Drama,category_Family,category_Foreign,category_Games,category_Horror,category_Music,category_New,category_Sci-Fi,category_Sports,category_Travel
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
188,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
189,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
190,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Re-concatenate everything

In [21]:
X_train_clean = pd.concat([X_train_norm, encoded_train_df], axis = 1)

In [22]:
X_test_clean = pd.concat([X_test_norm, encoded_test_df], axis = 1)

## 5. Create a logistic regression model to predict this variable from the cleaned data.

In [23]:
from sklearn.linear_model import LogisticRegression
classification = LogisticRegression(random_state=420, solver='lbfgs').fit(X_train_clean, y_train)

In [24]:
predictions = classification.predict(X_test_clean)
classification.score(X_test_clean, y_test)

0.7395833333333334

## 6. Evaluate the results.

In [25]:
print(y_test.value_counts())

True     142
False     50
Name: rented_last_month, dtype: int64


In [26]:
pd.Series(predictions).value_counts()

True    192
dtype: int64

### Not great - the predictions are only 'True'. I'll try the KNN method:

In [47]:
from sklearn import neighbors
clf = neighbors.KNeighborsClassifier(n_neighbors=5, weights='uniform')
clf.fit(X_train_clean, y_train)
predictions_clf = clf.predict(X_test_clean)
clf.score(X_test_clean, y_test)

0.6666666666666666

In [48]:
type(clf)

sklearn.neighbors._classification.KNeighborsClassifier

In [49]:
pd.Series(predictions_clf).value_counts()

True     162
False     30
dtype: int64

That *looks* better in terms of numerical distribution, but the score tells me that it's predicting the wrong ones. 

From fiddling around with the k number of neighbors, I figured out that the score gets higher the more neighbors I add - but this is because it guesses 'True' more often in those cases. In other words: guessing 'True' 100% of the time gives the highest possible score here.

I'll check what the original distribution of True and False was:

In [53]:
data['rented_last_month'].value_counts()

True     686
False    272
Name: rented_last_month, dtype: int64

That does not look all that skewed - it feels like a better prediction *might* be possible. 

However, I also know that this is a synthetic dataset, and I am starting to suspect that there are no real correlations between *any* of the features.

(This is also, for example, why all of the rentals take place in 2005, while all of the films are released in 2006. Time travel, or synthetic data?) 